The Social Force Model (Helbing & Molnár, 1995) is a physics-inspired model
for simulating pedestrian dynamics. Each agent (person) is influenced by:

1. A "desired force": pushes the agent toward their target at desired speed.
2. Repulsive forces: from walls and other agents to avoid collisions.
3. Friction/damping: to stabilize motion.

The equation of motion for pedestrian i is:

m_i * d²r_i/dt² = F_i^desired + Σ_j F_ij^repulsion + Σ_w F_iw^wall + ξ_i(t)

Where:
- r_i: position of agent i
- m_i: mass (usually 1)
- F_i^desired = m_i * (v_i^0 * e_i^0 - v_i) / τ_i
    - v_i^0: desired speed
    - e_i^0: desired direction (unit vector toward goal)
    - v_i: current velocity
    - τ_i: relaxation time (~0.5 s)
- F_ij^repulsion: exponential repulsion from other agents
- F_iw^wall: repulsion from walls (not modeled here for simplicity)
- ξ_i(t): noise (optional)

We'll implement a simplified 2D version without walls.
[link text](https://)

In [ ]:
# Install required packages (if needed)
!pip install matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
# ==============================
# 2. Core Implementation
# ==============================

class SocialForceModel:
    def __init__(self, num_agents, dt=0.05):
        self.N = num_agents
        self.dt = dt  # time step
        self.mass = 1.0
        self.relaxation_time = 0.5  # seconds
        self.A = 2000.0  # repulsion strength (N)
        self.B = 0.08    # repulsion range (m)
        self.k = 1.2e5   # body compression (not used here)
        self.kappa = 2.4e5  # friction (not used here)

        # Initialize positions and velocities
        self.pos = np.random.uniform(-5, 5, (self.N, 2))
        self.vel = np.zeros((self.N, 2))
        self.goal = np.random.uniform(-8, 8, (self.N, 2))

        # Clip goals to be away from start
        for i in range(self.N):
            while np.linalg.norm(self.goal[i] - self.pos[i]) < 2:
                self.goal[i] = np.random.uniform(-8, 8, 2)

        self.desired_speed = np.full(self.N, 1.3)  # m/s (average walking speed)

    def compute_forces(self):
        forces = np.zeros((self.N, 2))

        # Desired force
        for i in range(self.N):
            direction = self.goal[i] - self.pos[i]
            norm = np.linalg.norm(direction)
            if norm > 1e-6:
                e0 = direction / norm
            else:
                e0 = np.zeros(2)
            desired_force = (self.mass / self.relaxation_time) * (
                self.desired_speed[i] * e0 - self.vel[i]
            )
            forces[i] += desired_force

        # Agent-agent repulsion
        for i in range(self.N):
            for j in range(self.N):
                if i == j:
                    continue
                diff = self.pos[i] - self.pos[j]
                distance = np.linalg.norm(diff)
                if distance < 1e-6:
                    continue
                direction = diff / distance
                # Exponential repulsion
                repulsion = self.A * np.exp((self.B - distance) / self.B) * direction
                forces[i] += repulsion

        return forces

    def step(self):
        forces = self.compute_forces()
        acc = forces / self.mass
        self.vel += acc * self.dt
        self.pos += self.vel * self.dt

In [ ]:
# ==============================
# 3. Visualization Helper
# ==============================

def animate_scenario(model, title="Crowd Simulation", frames=300, interval=50):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    ax.set_title(title)
    ax.set_xlabel("X (m)")
    ax.set_ylabel("Y (m)")
    ax.grid(True, linestyle='--', alpha=0.5)

    # Plot goals as stars
    goal_plot = ax.scatter(model.goal[:, 0], model.goal[:, 1], c='gold', marker='*', s=100, label='Goals')

    # Initialize agent positions
    agents, = ax.plot([], [], 'bo', markersize=8, label='Agents')
    trajectories = [ax.plot([], [], 'b-', alpha=0.3)[0] for _ in range(model.N)]
    history = [[] for _ in range(model.N)]

    def init():
        agents.set_data([], [])
        for traj in trajectories:
            traj.set_data([], [])
        return [agents] + trajectories

    def update(frame):
        model.step()
        agents.set_data(model.pos[:, 0], model.pos[:, 1])

        # Update trajectories
        for i in range(model.N):
            history[i].append(model.pos[i].copy())
            if len(history[i]) > 50:  # keep last 50 steps
                history[i].pop(0)
            traj_x = [p[0] for p in history[i]]
            traj_y = [p[1] for p in history[i]]
            trajectories[i].set_data(traj_x, traj_y)

        return [agents] + trajectories

    anim = FuncAnimation(fig, update, frames=frames, init_func=init,
                         blit=True, interval=interval, repeat=False)
    plt.legend()
    plt.close(fig)  # Prevent static plot
    return HTML(anim.to_jshtml())


In [ ]:
# ==============================
# 4. Scenario 1: Random Walkers
# ==============================
print("Scenario 1: Random Goals — Agents move to random destinations")
model1 = SocialForceModel(num_agents=10)
anim1 = animate_scenario(model1, title="Scenario 1: Random Goals", frames=200)
anim1

In [ ]:
# ==============================
# 5. Scenario 2: Crossing Flows
# ==============================
print("\nScenario 2: Two opposing crowds crossing")
model2 = SocialForceModel(num_agents=12)
# Left group: start on left, go right
model2.pos[:6] = np.column_stack((np.full(6, -8), np.linspace(-3.1, 2.9, 6)))
model2.goal[:6] = np.column_stack((np.full(6, 8), np.linspace(-3.1, 2.9, 6)))

# Right group: start on right, go left
model2.pos[6:] = np.column_stack((np.full(6, 8), np.linspace(-3, 3, 6)))
model2.goal[6:] = np.column_stack((np.full(6, -8), np.linspace(-3, 3, 6)))

anim2 = animate_scenario(model2, title="Scenario 2: Crossing Flows", frames=300)
anim2

In [ ]:
# ==============================
# 6. Scenario 3: Bottleneck / Evacuation
# ==============================
print("\nScenario 3: Evacuation through a narrow exit")
model3 = SocialForceModel(num_agents=15)
# Place agents in a room (left side)
angles = np.linspace(0, 2*np.pi, 15, endpoint=False)
model3.pos = np.column_stack((4 * np.cos(angles) - 5, 4 * np.sin(angles)))
# All go to a narrow exit at (5, 0)
model3.goal = np.full((15, 2), (6, 0))

anim3 = animate_scenario(model3, title="Scenario 3: Bottleneck Evacuation", frames=400)
anim3

In [ ]:
# ==============================
# 7. Scenario 4: Circular Motion (Roundabout)
# ==============================
print("\nScenario 4: Agents walking in a circle (emergent lane formation)")
model4 = SocialForceModel(num_agents=20)
# Place agents on a circle
R = 6
angles = np.linspace(0, 2*np.pi, 20, endpoint=False)
model4.pos = np.column_stack((R * np.cos(angles), R * np.sin(angles)))
# Each agent aims for the point 90° ahead (counterclockwise)
target_angles = angles + np.pi / 2
model4.goal = np.column_stack((R * np.cos(target_angles), R * np.sin(target_angles)))

anim4 = animate_scenario(model4, title="Scenario 4: Circular Flow", frames=500)
anim4